# Μηχανική Μάθηση ~ Εργασία 2

### Θοδωρής Τσιρπάνης (`dai19090`)

---

## Προετοιμασία

Εισάγουμε κάποιες βιβλιοθήκες που θα χρειαστούν σε ολόκληρο το notebook.

Αρκετός κώδικας είναι βασισμένος στον κώδικα των εργαστηρίων του μαθήματος, αλλά και της προηγούμενης εργασίας.

In [1]:
import numpy as np
np.set_printoptions(precision=3)
from IPython.display import display, Markdown

random_seed = 59

## Φόρτωση των δεδομένων

Για την φόρτωση των δεδομένων πρέπει πρώτα το αρχείο `letter-recognition.data` να ανέβει στο notebook.

Τοποθετούμε την πρώτη στήλη (τον στόχο) σε ξεχωριστό πίνακα και μετατρέπουμε τα γράμματα σε αριθμούς με ένα αντικείμενο `LabelEncoder`.

Τις υπόλοιπες στήλες τις φοτρώνουμε σε έναν πίνακα του NumPy.

In [2]:
import csv
from sklearn.preprocessing import LabelEncoder

data_X = []
data_y_str = []
with open('letter-recognition.data') as myfile:
  data = csv.reader(myfile, delimiter=',') 
  for dataline in data: 
    data_X.append(dataline[1:])
    data_y_str.append(dataline[0])

labelEnc = LabelEncoder()

data_y = labelEnc.fit_transform(data_y_str)
data_X = np.array(data_X).astype(float)

print("Data loaded.")
print(f"Dimensions: {len(data_X)} * {len(data_X[0])}")

Data loaded.
Dimensions: 20000 * 16


Χωρίζουμε τα δεδομένα μας σε training και test όπως μας υπέδειξε η εκφώνηση, και εμφανίζουμε μερικά στατιστικά θέσης και διασποράς γι' αυτά.

Παρατηρούμε ότι οι τα στατιστικά των στηλών των δεδομένων μας δε διαφέρουν σε τάξη μεγέθους.

In [3]:
data_X_train = data_X[:16000,:]
data_y_train = data_y[:16000]
data_X_test = data_X[16000:,:]
data_y_test = data_y[16000:]

def np_str_oneLine(x):
  return np.array_str(x, max_line_width=np.inf)

display(Markdown(f"""### Data statistics
||Training Data|Test Data|
|-|-|-|
|**Dimensions**|{len(data_X_train)} * {len(data_X_train[0])}|{len(data_X_test)} * {len(data_X_test[0])}
|**Mean of each column**|{np_str_oneLine(data_X_train.mean(axis = 0))}|{np_str_oneLine(data_X_test.mean(axis = 0))}|
|**Variance of each column**|{np_str_oneLine(data_X_train.var(axis = 0))}|{np_str_oneLine(data_X_test.var(axis = 0))}|
|**Max of each column**|{np_str_oneLine(data_X_train.max(axis = 0))}|{np_str_oneLine(data_X_test.max(axis = 0))}|
|**Min of each column**|{np_str_oneLine(data_X_train.min(axis = 0))}|{np_str_oneLine(data_X_test.min(axis = 0))}|
"""))

### Data statistics
||Training Data|Test Data|
|-|-|-|
|**Dimensions**|16000 * 16|4000 * 16
|**Mean of each column**|[4.02  7.029 5.117 5.366 3.5   6.893 7.512 4.627 5.17  8.287 6.471 7.927 3.049 8.344 3.682 7.796]|[4.037 7.062 5.14  5.399 3.529 6.918 7.452 4.634 5.212 8.264 6.385 7.936 3.035 8.319 3.731 7.821]|
|**Variance of each column**|[ 3.642 10.915  4.013  5.119  4.806  4.119  5.447  7.352  5.687  6.178  6.98   4.293  5.487  2.403  6.612  2.571]|[ 3.732 10.936  4.24   5.091  4.766  4.048  5.246  7.039  5.59   6.247  6.685  4.472  5.255  2.351  6.499  2.797]|
|**Max of each column**|[15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.]|[14. 15. 14. 14. 15. 15. 15. 15. 15. 15. 15. 15. 15. 14. 15. 15.]|
|**Min of each column**|[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]|[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0.]|


Ορίζουμε μερικές συναρτήσεις που θα χρησιμοποιηθούν για την εκτέλεση και την αξιολόγηση του κάθε μοντέλου.

> Απαντώντας στη διόρθωση της προηγούμενης εργασίας, επίτηδες χρησιμοποιήθηκαν οι global μεταβλητές των δεδομένων ακριβώς για την ελαχιστοποίηση του επαναλαμβανόμενου κώδικα.

In [4]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

def make_pipeline(modelName, model):
  return Pipeline(steps=[
    ("scaler", MinMaxScaler()),
    (modelName, model)
  ])

def trainCV(model, params):
  grid = GridSearchCV(estimator=model, cv=5, param_grid=params, verbose=4)
  grid.fit(data_X_train, data_y_train)
  return grid

## Εκπαίδευση

Για κάθε μοντέλο που θα εκπαιδεύσουμε, υπάρχουν δύο κελλιά στο notepad· ένα για την εκπαίδευσή του και ένα για την εμφάνιση των αποτελεσμάτων. Αυτό μας επιτρέπει να αλλάζουμε τα στοιχεία του μοντέλου που θέλουμε να εμφανιστούν χωρίς να τα ξαναεκπαιδεύσουμε.

### Λογιστική Παλινδρόμηση

Χρησιμοποιήσαμε τον λύτη `saga` επειδή είναι πολύ πιο γρήγορος από τον προεπιλεγμένο, ο οποίος και δυσκολευόταν να συγκλίνει.

Επίσης η εκφώνηση της άσκησης ζητά να παραμετροποιήσουμε την υπερπαράμετρο $λ$, αλλά το sklearn μας δίνει την παράμετρο $C$ που σύμφωνα με την τεκμηρίωσή του είναι η αντίστροφη της $λ$. Προσαρμόζουμε ανάλογα την δήλωση του εύρους των υπερπαραμέτρων και την εμφάνισή τους.

In [5]:
from sklearn.linear_model import LogisticRegression

params = {
    "logistic__C": np.logspace(2, -3, 6)
}

model_logRegr = LogisticRegression(random_state=random_seed, solver="saga", tol=0.01)
pipeline_logRegr = make_pipeline("logistic", model_logRegr)
trained_logRegr = trainCV(pipeline_logRegr, params)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END .................logistic__C=100.0;, score=0.770 total time=   1.1s
[CV 2/5] END .................logistic__C=100.0;, score=0.769 total time=   1.1s
[CV 3/5] END .................logistic__C=100.0;, score=0.769 total time=   1.1s
[CV 4/5] END .................logistic__C=100.0;, score=0.762 total time=   1.1s
[CV 5/5] END .................logistic__C=100.0;, score=0.777 total time=   1.1s
[CV 1/5] END ..................logistic__C=10.0;, score=0.768 total time=   0.8s
[CV 2/5] END ..................logistic__C=10.0;, score=0.765 total time=   0.8s
[CV 3/5] END ..................logistic__C=10.0;, score=0.768 total time=   0.9s
[CV 4/5] END ..................logistic__C=10.0;, score=0.757 total time=   0.8s
[CV 5/5] END ..................logistic__C=10.0;, score=0.774 total time=   0.8s
[CV 1/5] END ...................logistic__C=1.0;, score=0.746 total time=   0.5s
[CV 2/5] END ...................logistic__C=1.0;,

In [6]:
display(Markdown(
    f"""### Logistic Regression
**Best score**: {trained_logRegr.best_score_:.3f}<br>
**Best $λ$**: {1.0 / trained_logRegr.best_params_["logistic__C"]}"""
))

### Logistic Regression
**Best score**: 0.769<br>
**Best $λ$**: 0.01

### Γραμμικό SVM

In [36]:
from sklearn.svm import SVC

params = {
    "linear_svm__C": np.logspace(3, 0, 4)
}

model_linearSvm = SVC(kernel="linear", tol=0.1)
pipeline_linearSvm = make_pipeline("linear_svm", model_linearSvm)
trained_linearSvm = trainCV(pipeline_linearSvm, params)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ..............linear_svm__C=1000.0;, score=0.858 total time=   5.9s
[CV 2/5] END ..............linear_svm__C=1000.0;, score=0.846 total time=   5.7s
[CV 3/5] END ..............linear_svm__C=1000.0;, score=0.851 total time=   5.9s
[CV 4/5] END ..............linear_svm__C=1000.0;, score=0.841 total time=   5.7s
[CV 5/5] END ..............linear_svm__C=1000.0;, score=0.857 total time=   5.9s
[CV 1/5] END ...............linear_svm__C=100.0;, score=0.859 total time=   3.0s
[CV 2/5] END ...............linear_svm__C=100.0;, score=0.849 total time=   2.9s
[CV 3/5] END ...............linear_svm__C=100.0;, score=0.850 total time=   3.0s
[CV 4/5] END ...............linear_svm__C=100.0;, score=0.838 total time=   2.9s
[CV 5/5] END ...............linear_svm__C=100.0;, score=0.859 total time=   3.0s
[CV 1/5] END ................linear_svm__C=10.0;, score=0.853 total time=   2.6s
[CV 2/5] END ................linear_svm__C=10.0;,

In [37]:
display(Markdown(
    f"""### Linear SVM
**Best score**: {trained_linearSvm.best_score_:.3f}<br>
**Best $C$**: {trained_linearSvm.best_params_["linear_svm__C"]}"""
))

### Linear SVM
**Best score**: 0.851<br>
**Best $C$**: 100.0

### k-Πλησιέστεροι Γείτονες

In [9]:
from sklearn.neighbors import KNeighborsClassifier

params = {
  "knn__weights": ["uniform", "distance"],
  "knn__n_neighbors": range(1,8)
}

model_knn = KNeighborsClassifier()
pipeline_knn = make_pipeline("knn", model_knn)
trained_knn = trainCV(pipeline_knn, params)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END knn__n_neighbors=1, knn__weights=uniform;, score=0.949 total time=   0.7s
[CV 2/5] END knn__n_neighbors=1, knn__weights=uniform;, score=0.947 total time=   0.5s
[CV 3/5] END knn__n_neighbors=1, knn__weights=uniform;, score=0.953 total time=   0.5s
[CV 4/5] END knn__n_neighbors=1, knn__weights=uniform;, score=0.949 total time=   0.5s
[CV 5/5] END knn__n_neighbors=1, knn__weights=uniform;, score=0.953 total time=   0.5s
[CV 1/5] END knn__n_neighbors=1, knn__weights=distance;, score=0.949 total time=   0.4s
[CV 2/5] END knn__n_neighbors=1, knn__weights=distance;, score=0.947 total time=   0.4s
[CV 3/5] END knn__n_neighbors=1, knn__weights=distance;, score=0.953 total time=   0.4s
[CV 4/5] END knn__n_neighbors=1, knn__weights=distance;, score=0.949 total time=   0.4s
[CV 5/5] END knn__n_neighbors=1, knn__weights=distance;, score=0.953 total time=   0.4s
[CV 1/5] END knn__n_neighbors=2, knn__weights=uniform;, score=0.

In [10]:
display(Markdown(
    f"""### k-Nearest Neighbors
**Best score**: {trained_knn.best_score_:.3f}<br>
**Best $k$**: {trained_knn.best_params_["knn__n_neighbors"]}<br>
**Best weighting strategy**: {trained_knn.best_params_["knn__weights"]}"""
))

### k-Nearest Neighbors
**Best score**: 0.951<br>
**Best $k$**: 4<br>
**Best weighting strategy**: distance

### SVM Συνάρτησης Ακτινικής Βάσης

In [11]:
from sklearn.svm import SVC

params = {
    "rbf_svm__C": np.logspace(0, 3, 4),
    "rbf_svm__gamma": np.logspace(1, -1, 6)
}

model_rbfSvm = SVC(tol=0.01)
pipeline_rbfSvm = make_pipeline("rbf_svm", model_rbfSvm)
trained_rbfSvm = trainCV(pipeline_rbfSvm, params)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END rbf_svm__C=1.0, rbf_svm__gamma=10.0;, score=0.970 total time=   7.9s
[CV 2/5] END rbf_svm__C=1.0, rbf_svm__gamma=10.0;, score=0.965 total time=   7.8s
[CV 3/5] END rbf_svm__C=1.0, rbf_svm__gamma=10.0;, score=0.973 total time=   7.8s
[CV 4/5] END rbf_svm__C=1.0, rbf_svm__gamma=10.0;, score=0.958 total time=   7.8s
[CV 5/5] END rbf_svm__C=1.0, rbf_svm__gamma=10.0;, score=0.964 total time=   7.8s
[CV 1/5] END rbf_svm__C=1.0, rbf_svm__gamma=3.9810717055349722;, score=0.956 total time=   5.3s
[CV 2/5] END rbf_svm__C=1.0, rbf_svm__gamma=3.9810717055349722;, score=0.956 total time=   5.3s
[CV 3/5] END rbf_svm__C=1.0, rbf_svm__gamma=3.9810717055349722;, score=0.955 total time=   5.3s
[CV 4/5] END rbf_svm__C=1.0, rbf_svm__gamma=3.9810717055349722;, score=0.948 total time=   5.3s
[CV 5/5] END rbf_svm__C=1.0, rbf_svm__gamma=3.9810717055349722;, score=0.951 total time=   5.4s
[CV 1/5] END rbf_svm__C=1.0, rbf_svm__gamma=1.58

In [12]:
display(Markdown(
    f"""### RBF SVM
**Best score**: {trained_rbfSvm.best_score_:.3f}<br>
**Best $C$**: {trained_rbfSvm.best_params_["rbf_svm__C"]}<br>
**Best $γ$**: {trained_rbfSvm.best_params_["rbf_svm__gamma"]}"""
))

### RBF SVM
**Best score**: 0.974<br>
**Best $C$**: 10.0<br>
**Best $γ$**: 10.0

### Τυχαίο Δάσος

Για τους αλγορίθμους βασισμένους σε ensembles δεν δώθηκαν υπαρπαράμετροι για cross-validation, αλλά πραγματοποιήθηκε παρ' όλα αυτά για διαφορετικά πλήθη δέντρων.

In [13]:
from sklearn.ensemble import RandomForestClassifier

params = {
  "random_forest__n_estimators": [50, 100, 150, 200]
}

model_randomForest = RandomForestClassifier(random_state= random_seed)
pipeline_randomForest = make_pipeline("random_forest", model_randomForest)
trained_randomForest = trainCV(pipeline_randomForest, params)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ....random_forest__n_estimators=50;, score=0.959 total time=   1.0s
[CV 2/5] END ....random_forest__n_estimators=50;, score=0.956 total time=   1.0s
[CV 3/5] END ....random_forest__n_estimators=50;, score=0.957 total time=   1.0s
[CV 4/5] END ....random_forest__n_estimators=50;, score=0.945 total time=   1.0s
[CV 5/5] END ....random_forest__n_estimators=50;, score=0.957 total time=   1.0s
[CV 1/5] END ...random_forest__n_estimators=100;, score=0.965 total time=   1.9s
[CV 2/5] END ...random_forest__n_estimators=100;, score=0.960 total time=   1.9s
[CV 3/5] END ...random_forest__n_estimators=100;, score=0.959 total time=   1.9s
[CV 4/5] END ...random_forest__n_estimators=100;, score=0.948 total time=   1.9s
[CV 5/5] END ...random_forest__n_estimators=100;, score=0.961 total time=   1.9s
[CV 1/5] END ...random_forest__n_estimators=150;, score=0.966 total time=   2.8s
[CV 2/5] END ...random_forest__n_estimators=150;,

In [14]:
display(Markdown(
    f"""### Random Forest
**Best score**: {trained_randomForest.best_score_:.3f}<br>
**Best number of trees**: {trained_randomForest.best_params_["random_forest__n_estimators"]}"""
))

### Random Forest
**Best score**: 0.959<br>
**Best number of trees**: 200

### Ακραία Τυχαιοποιημένα Δέντρα

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

params = {
  "extra_trees__n_estimators": [50, 100, 150, 200]
}

model_extraTrees = ExtraTreesClassifier()
pipeline_extraTrees = make_pipeline("extra_trees", model_extraTrees)
trained_extraTrees = trainCV(pipeline_extraTrees, params)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ......extra_trees__n_estimators=50;, score=0.967 total time=   0.8s
[CV 2/5] END ......extra_trees__n_estimators=50;, score=0.966 total time=   0.8s
[CV 3/5] END ......extra_trees__n_estimators=50;, score=0.965 total time=   0.8s
[CV 4/5] END ......extra_trees__n_estimators=50;, score=0.955 total time=   0.8s
[CV 5/5] END ......extra_trees__n_estimators=50;, score=0.964 total time=   0.8s
[CV 1/5] END .....extra_trees__n_estimators=100;, score=0.968 total time=   1.6s
[CV 2/5] END .....extra_trees__n_estimators=100;, score=0.969 total time=   1.6s
[CV 3/5] END .....extra_trees__n_estimators=100;, score=0.968 total time=   1.6s
[CV 4/5] END .....extra_trees__n_estimators=100;, score=0.957 total time=   1.6s
[CV 5/5] END .....extra_trees__n_estimators=100;, score=0.971 total time=   1.6s
[CV 1/5] END .....extra_trees__n_estimators=150;, score=0.970 total time=   2.5s
[CV 2/5] END .....extra_trees__n_estimators=150;,

In [16]:
display(Markdown(
    f"""### Extra Trees
**Best score**: {trained_extraTrees.best_score_:.3f}<br>
**Best number of trees**: {trained_extraTrees.best_params_["extra_trees__n_estimators"]}"""
))

### Extra Trees
**Best score**: 0.968<br>
**Best number of trees**: 200

## Ανάλυση Αποτελεσμάτων

In [26]:
all_models = [
  ("Logistic Regression", trained_logRegr),
  ("Linear SVM", trained_linearSvm),
  ("k-Nearest Neighbors", trained_knn),
  ("RBF SVM", trained_rbfSvm),
  ("Random Forest", trained_randomForest),
  ("Extra Trees", trained_extraTrees)
]

report = """|Model name|Precision|Recall|F1-score|
|-|-|-|-|
"""

for (name, model) in all_models:
  y_test_pred = model.predict(data_X_test)
  report += f"**{name}**|"
  report += f"{precision_score(data_y_test, y_test_pred, average='weighted'):.2%}|"
  report += f"{recall_score(data_y_test, y_test_pred, average='weighted'):.2%}|"
  report += f"{f1_score(data_y_test, y_test_pred, average='weighted'):.2%}\n"

display(Markdown(report))

|Model name|Precision|Recall|F1-score|
|-|-|-|-|
**Logistic Regression**|77.03%|77.12%|76.91%
**Linear SVM**|84.43%|84.12%|84.09%
**k-Nearest Neighbors**|95.90%|95.85%|95.85%
**RBF SVM**|97.89%|97.88%|97.87%
**Random Forest**|96.40%|96.33%|96.33%
**Extra Trees**|97.38%|97.32%|97.33%


Εμφανίζουμε το `classification_report` για το μοντέλο RBF SVM. Για να μετατρέψουμε τις αριθμητικές τιμές του μοντέλου σε γράμματα, χρησιμοποιούμε το αντικείμενο `LabelEncoder` που δημιουργήσαμε στην αρχή.

In [18]:
from sklearn.metrics import classification_report

y_test_pred = trained_rbfSvm.predict(data_X_test)
print(classification_report(data_y_test, y_test_pred, target_names=labelEnc.inverse_transform(range(0,26))))

              precision    recall  f1-score   support

           A       0.99      0.99      0.99       156
           B       0.94      0.99      0.96       136
           C       0.99      0.99      0.99       142
           D       0.96      0.98      0.97       167
           E       0.98      0.95      0.97       152
           F       0.99      0.97      0.98       153
           G       0.98      0.99      0.98       164
           H       0.94      0.91      0.93       151
           I       0.97      0.98      0.97       165
           J       0.97      0.96      0.97       148
           K       0.95      0.95      0.95       146
           L       0.99      0.97      0.98       157
           M       0.98      0.99      0.99       144
           N       0.99      0.97      0.98       166
           O       0.98      0.99      0.99       139
           P       0.99      0.98      0.99       168
           Q       1.00      0.98      0.99       168
           R       0.96    

### Συμπεράσματα

* Το χειρότερο μοντέλο αποδείχτηκε αυτό της λογιστικής παλινδρόμησης. Μετά από αυτό βρίσκεται το γραμμικό SVM και στην κορυφή βρίσκονται κοντά μεταξύ τους τα τέσσερα μη γραμμικά μοντέλα. Yποδεικνύεται ότι η σχέση μεταξύ των $X$ και $y$ δεν είναι γραμμική.
* Η αρχική υλοποίηση του γραμμικού SVM βασιζόταν στην κλάση `LinearSVC`, ήταν εξαιρετικά αργή και τα αποτελέσματά της ήταν κοντά στην λογιστική παλινδρόμηση. Επειδή δεν υποστηρίζει την εξαγωγή των διανυσμάτων υποστήριξης, άλλαξε στην κλάση `SVC` με γραμμικό πυρήνα, και η ταχύτητά της εκτινάχθηκε, με τα αποτελέσματά της να βελτιώνονται, πράγμα περίεργο καθώς το sklearn προτείνει την πρώτη κλάση για γραμμικά SVM.
* Η ταχύτητα εκπαίδευσής των μοντέλων είναι ικανοποιητική, με εξαίρεση το RBF SVM επειδή ζητήθηκαν πολλοί συνδυασμοί υπερπαραμέτρων.
* Το μοντέλο RBF SVM δυσκολεύεται λίγο περισσότερο να αναγνωρίσει τον χαρακτήρα `H` σε σχέση με τους άλλους, αλλά κανένας χαρακτήρας δεν έχει σε κάποιον δείκτη επιτυχίας τιμή κάτω του 90%.

## Επιπλέον ερωτήσεις

1. __Τα καλύτερα αποτελέσματα για κάθε \[SVM\] πυρήνα προέκυψαν με ισχυρό ή με ασθενές
regularization;__ Η ισχύς του regularization προκύπτει από την τιμή της παραμέτρου $C$, η οποία [σύμφωνα με την τεκμηρίωση του sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), η τιμή του $C$ είναι αντιστρόφως ανάλογη της ισχύος του regularization. Οπότε με το $C$ στον γραμμικό πυρήνα να είναι ίσο με $100$ και στον RBF ίσο με $10$, και λαμβάνοντας υπ' όψιν ότι οι τιμές που δοκιμάστηκαν κυμαίνονται μεταξύ του $1$ και του $1000$, τα καλύτερα αποτελέσματα για τον γραμμικό πυρήνα προέκυψαν με _μέτρια ασθενές_ regularization, και για τον πυρήνα RBF προέκυψαν με _μέτρια ισχυρό_ regularization.

2. __Σε πόσα διανύσματα υποστήριξης για κάθε κλάση κατέληξε ο αλγόριθμος του SVM
γραμμικού πυρήνα και σε πόσα το RBF SVM;__ Αυτό μπορούμε να το ανακαλύψουμε εκτελώντας τον παρακάτω κώδικα:

In [39]:
def get_n_support(model, stepName):
  return model.best_estimator_.named_steps[stepName].n_support_

display(Markdown(f"""**Linear SVM:** {get_n_support(trained_linearSvm, "linear_svm")}<br>
**RBF SVM**: {get_n_support(trained_rbfSvm, "rbf_svm")}"""))

**Linear SVM:** [117 348 214 231 292 285 439 484 208 218 273 179 163 179 445 194 330 330
 438 217 168 213 136 261 213 230]<br>
**RBF SVM**: [213 388 283 372 361 347 363 385 210 271 384 218 328 328 320 272 343 369
 391 275 309 267 268 353 332 268]